In [1]:
import quippy
import ase
from ase.calculators.calculator import all_changes

import logging
logging.basicConfig(level=logging.WARNING)

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import os
import sys

In [4]:
sys.path.insert(0,"/home/es732/programs/function_modules/")
import mbx_functions

In [5]:
dirpath = os.getcwd()
print dirpath

/home/es732/programs/mbx_tests


In [6]:
from collections import OrderedDict

In [7]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [8]:
version_list=["mbx_pbc_no_ww_www_mww"]

In [9]:
dict_version_d_virial=OrderedDict([(version,
                                    OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                   for version in version_list])

In [10]:
dict_version_model_virial=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [11]:
def calculate_numerical_virial_diag(pot, in_atoms, d=1e-2):
    ## modified from the function calculate_numerical_stress() at ase/calculators/calculator.py 
    """Calculate numerical stress using finite difference."""

    atoms=in_atoms.copy()
    stress = np.zeros((3, 3), dtype=float)

    cell = atoms.cell.copy()
    V = atoms.get_volume()
    for i in range(3):
        x = np.eye(3)
        x[i, i] += d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eplus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        x[i, i] -= 2 * d
        atoms.set_cell(np.dot(cell, x), scale_atoms=True)
        atoms.set_calculator(pot)
        eminus = atoms.get_potential_energy()#force_consistent=True)
        pot.reset()

        stress[i, i] = (eplus - eminus) / (2 * d * V)
        x[i, i] += d
    atoms.set_cell(cell, scale_atoms=True)
    virial= (-stress*atoms.get_volume())
    return virial

In [12]:
this_root="/home/es732/programs/mbx_tests/single_point/pbc/clathrate/"

In [13]:
this_struct="phase_clathrate_2x2x2"
this_at_tmp=quippy.Atoms(this_root+"11.74290/"+this_struct+".xyz")
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

## diagonal_virial=F, no PIPs except mw 2B

In [14]:
this_version=version_list[0]
print this_version

mbx_pbc_no_ww_www_mww


In [15]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [16]:
at= this_at_tmp.copy()

In [17]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial[this_version] = np.array( at.params["virial"])
print dict_version_model_virial[this_version]

[[-1146.46446482     2.58750954     4.03548057]
 [    2.58750954 -1163.34812123     4.75634431]
 [    4.03548057     4.75634431 -1158.59329643]]


In [18]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

In [19]:
version_list

['mbx_pbc_no_ww_www_mww']

## Errors in derivative virial / Virial from model (only diagonal)

In [20]:
this_version="mbx_pbc_no_ww_www_mww"
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial[this_version][tmp_d] -dict_version_model_virial[this_version])/dict_version_model_virial[this_version])

d = 0.01
[0.00377235 0.00380018 0.00360712]
d = 0.001
[0.00416036 0.00417072 0.00396165]
d = 0.0001
[0.00416433 0.00417446 0.00396527]


In [21]:
## 3x3x3 cell

In [22]:
d_list=[0.1,0.01,0.001,0.0001,0.00001,0.000001]

In [23]:
version_list=["mbx_pbc_no_ww_www_mww"]

In [24]:
dict_version_d_virial_3x3x3=OrderedDict([(version,
                                    OrderedDict([(d, [[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                                 for d in d_list]))
                                   for version in version_list])

In [25]:
dict_version_model_virial_3x3x3=OrderedDict([(version,[[np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN], [np.NaN, np.NaN, np.NaN]])
                                   for version in version_list])

In [26]:
this_root="/home/es732/programs/mbx_tests/single_point/pbc/clathrate/"

In [27]:
this_struct="phase_clathrate_3x3x3"
this_at_tmp=quippy.Atoms(this_root+"11.74290/"+this_struct+".xyz")
dict_pot=mbx_functions.create_mbx_potential_variables(this_at_tmp)

## diagonal_virial=F, no PIPs except mw 2B

In [28]:
this_version=version_list[0]
print this_version

mbx_pbc_no_ww_www_mww


In [29]:
pot_MBX = quippy.Potential("IP MBXPBC diagonal_virial=F n_monomers_types={"+
                           dict_pot["n_monomers_types"]+"} nmon="+str(dict_pot["nmon"])+
                           " json_file="+this_version+".json",param_str="")

In [30]:
at= this_at_tmp.copy()

In [31]:
pot_MBX.calc(at,virial=True)
dict_version_model_virial_3x3x3[this_version] = np.array( at.params["virial"])
print dict_version_model_virial_3x3x3[this_version]

[[-3869.31606811     8.73278381    13.61993547]
 [    8.73278381 -3926.29330327    16.05295443]
 [   13.61993547    16.05295443 -3910.25204506]]


In [32]:
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    dict_version_d_virial_3x3x3[this_version][tmp_d] = calculate_numerical_virial_diag(pot=pot_MBX,in_atoms=this_at_tmp.copy(),d=tmp_d)
    

## Errors in derivative virial / Virial from model (only diagonal)

In [33]:
this_version="mbx_pbc_no_ww_www_mww"
for tmp_d in [0.01,0.001,0.0001]:#,0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001]:
    print "d =",tmp_d
    print np.diag((dict_version_d_virial_3x3x3[this_version][tmp_d] -dict_version_model_virial_3x3x3[this_version])/dict_version_model_virial_3x3x3[this_version])

d = 0.01
[0.00377235 0.00380018 0.00360712]
d = 0.001
[0.00416036 0.00417072 0.00396165]
d = 0.0001
[0.00416433 0.00417447 0.00396527]
